# 绘图-10折同源性与2018later 同源性
> 2025-06-20  
> Author: zhenkun.shi@tib.cas.cn

## 1. 导入必要的包

In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
import pandas as pd
import numpy as np
from scipy.interpolate import make_interp_spline
import plotly.graph_objects as go
import tools.bioFunctionLib as bfl

%load_ext autoreload
%autoreload 2

## 2. 读取数据

In [2]:
# ds_rcv
ds_train_fold10 = pd.read_feather(f'{cfg.DIR_DATASET}validation/fold1/train.feather')
ds_test_fold10 = pd.read_feather(f'{cfg.DIR_DATASET}validation/fold1/valid.feather')

# ds_rcp
ds_train_2018later = pd.read_feather(cfg.FILE_DS_TRAIN)
ds_test_2018later = pd.read_feather(cfg.FILE_DS_TEST)

## 3. 序列比对

In [3]:
blast_res_fold10 = bfl.getblast(train=ds_train_fold10[['uniprot_id', 'seq']], test=ds_test_fold10[['uniprot_id', 'seq']], k=1)
blast_res_fold10 = blast_res_fold10[['id','pident']].sort_values(by='pident').reset_index(drop=True)
blast_res_fold10 = ds_test_fold10[['uniprot_id']].rename(columns={'uniprot_id': 'id'}).merge(blast_res_fold10, on='id', how='left').fillna(0).sort_values(by='pident').reset_index(drop=True)

blast_res_2018later = bfl.getblast(train=ds_train_2018later[['uniprot_id', 'seq']], test=ds_test_2018later[['uniprot_id', 'seq']], k=1)
blast_res_2018later = blast_res_2018later[['id','pident']].sort_values(by='pident').reset_index(drop=True)
blast_res_2018later = ds_test_2018later[['uniprot_id']].rename(columns={'uniprot_id': 'id'}).merge(blast_res_2018later, on='id', how='left').fillna(0).sort_values(by='pident').reset_index(drop=True)

## 4. 统计区间分布

In [4]:
def compute_pident_distribution(df, column='pident', bin_width=10, range_max=100, merge_threshold=30):
    """
    按百分比区间统计 pident 分布频数和占比（cat_prob），并将小于 merge_threshold 的区间合并。
    
    参数:
        df: 输入的 pandas DataFrame
        column: 要处理的列名，默认为 'pident'
        bin_width: 每个区间的宽度，默认为10
        range_max: 最大值上限（如100）
        merge_threshold: 小于该百分比的区间将合并为一个 bin（默认合并<30%）

    返回:
        pandas DataFrame，包含：
            - pident_range: 区间标签（合并后含 <xx%）
            - count: 每个区间内的频数
            - cat_prob: 每个区间的占比（百分比，保留两位小数，总和为100%）
    """
    bins = np.arange(0, range_max + bin_width, bin_width)
    labels = [f'{i}–{i + bin_width}' for i in range(0, range_max, bin_width)]

    df = df.copy()
    df['pident_range'] = pd.cut(
        df[column],
        bins=bins,
        labels=labels,
        right=True,              # 关键修改：右闭区间 [a, b]
        include_lowest=True      # 确保最小值也被包含
    )

    freq_series = df['pident_range'].value_counts().sort_index()

    result = pd.DataFrame({
        'pident_range': freq_series.index,
        'count': freq_series.values,
    })

    total = len(df)
    result['cat_prob'] = (result['count'] / total * 100).round(2)

    # 提取区间起始值
    range_start = result['pident_range'].str.extract(r'(\d+)', expand=False).astype(int)

    # 拆分高低区间
    low_bins = result[range_start < merge_threshold]
    high_bins = result[range_start >= merge_threshold]

    if not low_bins.empty:
        merged = pd.DataFrame({
            'pident_range': [f'<{merge_threshold}%'],
            'count': [low_bins['count'].sum()],
            'cat_prob': [round(low_bins['count'].sum() / total * 100, 2)]
        })
        result = pd.concat([merged, high_bins], ignore_index=True)

    return result

In [5]:
freq_10fold = compute_pident_distribution(blast_res_fold10)
freq_2018later = compute_pident_distribution(blast_res_2018later)

In [6]:
print('Sequence Smilarity Distribution on ds_rcv')
freq_10fold

Sequence Smilarity Distribution on ds_rcv


,pident_range,count,cat_prob
0,<30%,2767,5.44
1,30–40,1176,2.31
2,40–50,1996,3.92
3,50–60,2804,5.51
4,60–70,3810,7.49
5,70–80,4666,9.17
6,80–90,6429,12.64
7,90–100,27210,53.50


In [7]:
print('Sequence Smilarity Distribution on ds_rcp')
freq_2018later

Sequence Smilarity Distribution on ds_rcp


,pident_range,count,cat_prob
0,<30%,5167,38.23
1,30–40,1842,13.63
2,40–50,1386,10.26
3,50–60,1082,8.01
4,60–70,926,6.85
5,70–80,890,6.59
6,80–90,741,5.48
7,90–100,1481,10.96


### 5. 绘图

In [13]:
# 示例 x 轴（索引）
x_vals = list(range(len(freq_10fold)))

# 拟合趋势线
x_vals = list(range(len(freq_10fold)))
trend_10fold = np.polyval(np.polyfit(x_vals, freq_10fold['cat_prob'], deg=3), x_vals)
trend_2018 = np.polyval(np.polyfit(x_vals, freq_2018later['cat_prob'], deg=3), x_vals)

# 创建图表
fig = go.Figure()

# 添加柱状图（10-Fold，含数值）
fig.add_trace(go.Bar(
    x=freq_10fold['pident_range'],
    y=freq_10fold['cat_prob'],
    name='ds_rcv',
    marker_color='#8ECFC9',
    text=freq_10fold['cat_prob'],
    textposition='outside'
))

# 添加柱状图（Post-2018，含数值）
fig.add_trace(go.Bar(
    x=freq_2018later['pident_range'],
    y=freq_2018later['cat_prob'],
    name='ds_rcp',
    marker_color='#FA7F6F',
    text=freq_2018later['cat_prob'],
    textposition='outside'
))

# 添加趋势线（ds_rcv）
fig.add_trace(go.Scatter(
    x=freq_10fold['pident_range'],
    y=trend_10fold,
    mode='lines',
    name='Trend (ds_rcv)',
    line=dict(color='#0072B5', dash='dash')
))

# 添加趋势线（ds_rcp）
fig.add_trace(go.Scatter(
    x=freq_2018later['pident_range'],
    y=trend_2018,
    mode='lines',
    name='Trend (ds_rcp)',
    line=dict(color='#BC3C29', dash='dot')
))

# 图形样式设置
fig.update_layout(
    title='Cat_prob Distribution across Identity Intervals with Trendlines',
    xaxis_title='Identity Range (%)',
    yaxis=dict(
        title='cat_prob (%)',
        range=[0, 60],
        showline=True,
        linecolor='black',
        gridcolor='gray'
    ),
    barmode='group',  # 并排显示
    xaxis_tickangle=-45,
    width=1200,
    height=600,
    template='plotly_white'
)

fig.show()

## 6. 酶/非酶 占比

In [9]:
# 提取酶 / 非酶 的数量
enzyme_count = len(ds_test_2018later[ds_test_2018later.reaction_id != '-'])
non_enzyme_count = len(ds_test_2018later[ds_test_2018later.reaction_id == '-'])
total_count = enzyme_count + non_enzyme_count

# 数据和颜色
labels = ['Enzyme', 'Non-Enzyme']
values = [enzyme_count, non_enzyme_count]
colors = ['#8ECFC9', '#FA7F6F']

# 创建饼图（环形）
fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    marker=dict(colors=colors),
    textinfo='label+percent+value',
    hoverinfo='label+percent+value',
    hole=0.4  # 环形饼图
)])

# 添加中央注释（总数）
fig.update_layout(
    title=dict(
        text='Enzyme vs Non-Enzyme Distribution',
        x=0.5,
        font=dict(size=20)
    ),
    annotations=[dict(
        text=f'Total<br>{total_count:,}',  # 加逗号分隔
        x=0.5, y=0.5,
        font_size=16,
        showarrow=False
    )],
    showlegend=True,
    width=600,
    height=600,
    template='plotly_white'
)

fig.show()